# Bonus — Production Setup

Move from local docker-compose to production:
- **Backend store:** Supabase PostgreSQL
- **Artifact store:** Azure Blob Storage

All secrets are loaded from a `.env` file (see `.env.example` for the template).

## 1. Supabase PostgreSQL (backend store)

Connection string (use **session pooler**, port 5432):

```
postgresql+psycopg2://postgres.<REF>:<PASS>@aws-0-<REGION>.pooler.supabase.com:5432/postgres?sslmode=require
```

> **Do NOT use port 6543** (transaction pooler) — it breaks prepared statements that SQLAlchemy/MLflow relies on.

## 2. Azure Blob Storage (artifact store)

Artifact root format:
```
wasbs://mlflow-artifacts@<account>.blob.core.windows.net/
```

Requires: `pip install azure-storage-blob`

## 3. Load configuration from `.env`

In [3]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

backend_store_uri = os.environ["MLFLOW_BACKEND_STORE_URI"]
artifact_root = os.environ["MLFLOW_ARTIFACT_ROOT"]

# AZURE_STORAGE_CONNECTION_STRING is read automatically by azure-storage-blob
print(f"Backend store: {backend_store_uri[:40]}...")
print(f"Artifact root: {artifact_root}")

Backend store: postgresql+psycopg2://postgres:VN4AuFGT2...
Artifact root: wasbs://mlflow-artifacts@stmlflowfd2025.blob.core.windows.net/


## 4. Launch the MLflow server

From the repo root:

```bash
# Load env vars and start the server
set -a && source .env && set +a

mlflow server \
  --backend-store-uri "$MLFLOW_BACKEND_STORE_URI" \
  --default-artifact-root "$MLFLOW_ARTIFACT_ROOT" \
  --host 127.0.0.1 --port 5050
```

## 5. Validate the connection

In [4]:
import mlflow

PRODUCTION_URI = "http://localhost:5050"

mlflow.set_tracking_uri(PRODUCTION_URI)

client = mlflow.MlflowClient()
experiments = client.search_experiments()
print(f"Connected to {PRODUCTION_URI}")
print(f"Found {len(experiments)} experiment(s):")
for exp in experiments:
    print(f"  - {exp.name}")

Connected to http://localhost:5050
Found 1 experiment(s):
  - Default


In [5]:
# Smoke test: log a dummy run to verify backend + artifact store
mlflow.set_experiment("production-smoke-test")

with mlflow.start_run(run_name="smoke-test"):
    mlflow.log_param("test", "connection")
    mlflow.log_metric("dummy", 42.0)

    import tempfile
    with tempfile.NamedTemporaryFile(mode="w", suffix=".txt", delete=False) as f:
        f.write("artifact round-trip test")
        tmp_path = f.name
    mlflow.log_artifact(tmp_path)
    os.unlink(tmp_path)

print("Smoke test passed — backend and artifact store working.")

2026/02/23 23:19:11 INFO mlflow.tracking.fluent: Experiment with name 'production-smoke-test' does not exist. Creating a new experiment.


🏃 View run smoke-test at: http://localhost:5050/#/experiments/1/runs/0f8f4c6093964200a6a80cf0802e055c
🧪 View experiment at: http://localhost:5050/#/experiments/1
Smoke test passed — backend and artifact store working.


## 6. Browse artifacts in Azure Blob Storage

In [6]:
from azure.storage.blob import ContainerClient

connection_string = os.environ["AZURE_STORAGE_CONNECTION_STRING"]
container = ContainerClient.from_connection_string(connection_string, "mlflow-artifacts")

print("Artifacts in Azure Blob Storage:\n")
for blob in container.list_blobs():
    print(f"  {blob.name}  ({blob.size} bytes)")

Artifacts in Azure Blob Storage:

  1/0f8f4c6093964200a6a80cf0802e055c/artifacts/tmpvxi0gc1t.txt  (24 bytes)
